In [7]:
# ==========================================
# 1. 환경 설정 및 라이브러리 임포트
# ==========================================
import sys
import os
import pickle
import numpy as np
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 경로 설정 (본인 경로에 맞게 수정)
PROJECT_PATH = '/content/drive/MyDrive/DL_Project'
os.chdir(PROJECT_PATH)
sys.path.append(PROJECT_PATH)

try:
    from common.np import *
    from common.time_layers import TimeEmbedding, TimeLSTM, TimeDropout
    from common.layers import Affine, SoftmaxWithLoss, Sigmoid, Dropout
    from common.optimizer import Adam
    from utils import extract_utterances
    print("✅ 라이브러리 로드 성공.")
except ImportError as e:
    print(f"❌ 라이브러리 로드 실패: {e}")

# 시각화 설정
plt.style.use('seaborn-v0_8')

# ==========================================
# 2. 데이터 전처리 및 로더
# ==========================================
def load_vocab(train_csv, max_vocab=4000):
    df = pd.read_csv(train_csv)
    words = []
    print("⏳ Vocab 생성 중...")
    for _, row in df.iterrows():
        try:
            utts = extract_utterances(row['filename'], ['CHI'])
            text = " ".join([u.clean_text for u in utts]).lower()
            words.extend(text.split())
        except: continue
    counter = Counter(words)
    most_common = counter.most_common(max_vocab)
    vocab = {'<pad>': 0, '<unk>': 1}
    for i, (w, _) in enumerate(most_common):
        vocab[w] = i + 2
    print(f"✅ Vocab 크기: {len(vocab)}")
    return vocab

def load_data(csv_path, vocab, max_len=400):
    df = pd.read_csv(csv_path)
    x_data, t_data = [], []
    label_map = {'TD': 0, 'SLI': 1}
    print(f"📂 데이터 로딩: {csv_path}")
    for _, row in df.iterrows():
        try:
            utts = extract_utterances(row['filename'], ['CHI'])
            if not utts: continue
            text = " ".join([u.clean_text for u in utts]).lower()
            tokens = text.split()
            ids = [vocab.get(w, vocab['<unk>']) for w in tokens]

            # 길이 제한 (Truncation)
            if len(ids) > max_len:
                ids = ids[:max_len]

            # 패딩 (Padding)
            padded_ids = np.zeros(max_len, dtype=int)
            length = min(len(ids), max_len)
            padded_ids[:length] = ids[:length]

            x_data.append(padded_ids)
            t_data.append(label_map[row['group']])
        except: pass
    return np.array(x_data), np.array(t_data)

# ==========================================
# 3. 모델 정의 (Bi-LSTM + MLP)
# ==========================================
class TimeBiLSTM:
    def __init__(self, Wx1, Wh1, b1, Wx2, Wh2, b2, stateful=False):
        self.forward_lstm = TimeLSTM(Wx1, Wh1, b1, stateful)
        self.backward_lstm = TimeLSTM(Wx2, Wh2, b2, stateful)
        self.params = self.forward_lstm.params + self.backward_lstm.params
        self.grads = self.forward_lstm.grads + self.backward_lstm.grads

    def forward(self, xs):
        o1 = self.forward_lstm.forward(xs)
        xs_reverse = xs[:, ::-1, :]
        o2 = self.backward_lstm.forward(xs_reverse)
        o2 = o2[:, ::-1, :]
        return np.concatenate((o1, o2), axis=2)

    def backward(self, dhs):
        H = dhs.shape[2] // 2
        do1 = dhs[:, :, :H]
        do2 = dhs[:, :, H:]
        dxs1 = self.forward_lstm.backward(do1)
        do2_reverse = do2[:, ::-1, :]
        dxs2 = self.backward_lstm.backward(do2_reverse)
        dxs2 = dxs2[:, ::-1, :]
        return dxs1 + dxs2

class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, h):
        N, T, H = hs.shape
        hr = h.reshape(N, 1, H)
        t = np.sum(hs * hr, axis=2)
        t -= np.max(t, axis=1, keepdims=True)
        exp_t = np.exp(t)
        a = exp_t / np.sum(exp_t, axis=1, keepdims=True)
        ar = a.reshape(N, T, 1)
        c = np.sum(hs * ar, axis=1)
        self.cache = (hs, hr, a)
        return c

    def backward(self, dc):
        hs, hr, a = self.cache
        N, T, H = hs.shape
        ar = a.reshape(N, T, 1)
        d_hs = dc.reshape(N, 1, H) * ar
        da = np.sum(dc.reshape(N, 1, H) * hs, axis=2)
        dt = a * (da - np.sum(da * a, axis=1, keepdims=True))
        d_hs += dt.reshape(N, T, 1) * hr
        d_hr = np.sum(dt.reshape(N, T, 1) * hs, axis=1)
        dh = d_hr.reshape(N, H)
        return d_hs, dh

class BiAttentionMLPClassifier:
    def __init__(self, vocab_size, wordvec_size, hidden_size, class_size=2, dropout_ratio=0.5):
        V, D, H, C = vocab_size, wordvec_size, hidden_size, class_size
        rn = np.random.randn

        # 1. Embedding & Bi-LSTM
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx1 = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh1 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b1 = np.zeros(4 * H).astype('f')
        lstm_Wx2 = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh2 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b2 = np.zeros(4 * H).astype('f')

        # 2. MLP Head (4H -> H -> C)
        input_size = 4 * H
        W1 = (rn(input_size, H) / np.sqrt(input_size)).astype('f')
        b1 = np.zeros(H).astype('f')
        W2 = (rn(H, C) / np.sqrt(H)).astype('f')
        b2 = np.zeros(C).astype('f')

        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeBiLSTM(lstm_Wx1, lstm_Wh1, lstm_b1, lstm_Wx2, lstm_Wh2, lstm_b2)
        self.attention = Attention()
        self.dropout1 = TimeDropout(dropout_ratio)
        self.affine1 = Affine(W1, b1)
        self.activation = Sigmoid()
        self.dropout2 = Dropout(dropout_ratio)
        self.affine2 = Affine(W2, b2)
        self.loss_layer = SoftmaxWithLoss()

        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine1, self.affine2):
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        hs = self.dropout1.forward(hs)
        h_last = hs[:, -1, :]
        c = self.attention.forward(hs, h_last)
        out = np.concatenate((c, h_last), axis=1)

        out = self.affine1.forward(out)
        out = self.activation.forward(out)
        out = self.dropout2.forward(out)
        score = self.affine2.forward(out)
        return score

    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        dout = self.affine2.backward(dout)
        dout = self.dropout2.backward(dout)
        dout = self.activation.backward(dout)
        dout = self.affine1.backward(dout)

        N, H4 = dout.shape
        H2 = H4 // 2
        dc, dh_last = dout[:, :H2], dout[:, H2:]
        d_hs_att, dh_att = self.attention.backward(dc)
        dhs = d_hs_att
        dhs[:, -1, :] += (dh_last + dh_att)

        dhs = self.dropout1.backward(dhs)
        dhs = self.lstm.backward(dhs)
        self.embed.backward(dhs)
        return None

    def set_train_flg(self, train_flg=True):
        self.dropout1.train_flg = train_flg
        self.dropout2.train_flg = train_flg

# ==========================================
# 4. FGM (적대적 학습) 구현 - Numpy 버전
# ==========================================
class FGM:
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=0.05): # Epsilon을 작게 설정 (중요)
        # 임베딩 레이어의 가중치(W)에 노이즈 추가
        # model.embed.params[0]은 임베딩 행렬
        W = self.model.embed.params[0]
        grad = self.model.embed.grads[0]

        # 백업
        self.backup['embed_W'] = W.copy()

        # 노이즈 계산 (Gradient 방향)
        norm = np.sqrt(np.sum(grad**2))
        if norm != 0:
            r_at = epsilon * grad / norm
            W += r_at # 공격 적용

    def restore(self):
        # 원상 복구
        if 'embed_W' in self.backup:
            self.model.embed.params[0][...] = self.backup['embed_W']
        self.backup = {}

# ==========================================
# 5. 학습 설정 및 실행
# ==========================================
MAX_LEN = 300       # 안정적인 학습을 위해 적절한 길이 유지
BATCH_SIZE = 16
MAX_EPOCH = 30      # FGM 덕분에 과적합이 덜하므로 에폭을 늘림
HIDDEN_SIZE = 128
WORDVEC_SIZE = 100
DROPOUT_RATIO = 0.5
LEARNING_RATE = 0.01

train_csv = 'split/gillam_train.csv'
dev_csv = 'split/gillam_dev.csv'
test_csv = 'split/gillam_test.csv'

vocab = load_vocab(train_csv, max_vocab=4000)
x_train, t_train = load_data(train_csv, vocab, max_len=MAX_LEN)
x_dev, t_dev = load_data(dev_csv, vocab, max_len=MAX_LEN)
x_test, t_test = load_data(test_csv, vocab, max_len=MAX_LEN)

if np.__name__ == 'cupy':
    x_train, t_train = np.asarray(x_train), np.asarray(t_train)
    x_dev, t_dev = np.asarray(x_dev), np.asarray(t_dev)
    x_test, t_test = np.asarray(x_test), np.asarray(t_test)

# 모델 & FGM 초기화
model = BiAttentionMLPClassifier(len(vocab), WORDVEC_SIZE, HIDDEN_SIZE, dropout_ratio=DROPOUT_RATIO)
optimizer = Adam(lr=LEARNING_RATE)
fgm = FGM(model)

data_size = len(x_train)
max_iters = data_size // BATCH_SIZE
best_valid_acc = 0.0

print(f"\n🚀 학습 시작 (Bi-LSTM + MLP + R-Drop + FGM)")

for epoch in range(MAX_EPOCH):
    model.set_train_flg(True)
    idx = np.random.permutation(data_size)
    x_train = x_train[idx]
    t_train = t_train[idx]

    total_loss = 0
    loss_count = 0

    for i in range(max_iters):
        batch_x = x_train[i*BATCH_SIZE : (i+1)*BATCH_SIZE]
        batch_t = t_train[i*BATCH_SIZE : (i+1)*BATCH_SIZE]

        # 1. 일반 학습 (Clean Data)
        loss = model.forward(batch_x, batch_t)
        model.backward() # 기울기 계산

        # 2. 적대적 학습 (Adversarial Data - FGM)
        fgm.attack(epsilon=0.05) # 공격! (임베딩에 노이즈)

        loss_adv = model.forward(batch_x, batch_t) # 노이즈 낀 상태로 Loss 계산
        model.backward() # 기울기 누적 (Gradient Accumulation 효과)

        fgm.restore() # 복구

        # 3. 가중치 업데이트 (Clean + Adv 기울기 모두 반영)
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

    # 평가
    model.set_train_flg(False)
    model_score = model.predict(x_dev)
    pred = np.argmax(model_score, axis=1)
    acc = np.sum(pred == t_dev) / len(t_dev)

    avg_loss = total_loss / loss_count

    # 최고 모델 저장
    if acc > best_valid_acc:
        best_valid_acc = acc
        with open('best_model.pkl', 'wb') as f:
            pickle.dump(model.params, f)
        print(f"| Epoch {epoch+1:02} | Loss {avg_loss:.4f} | Dev Acc {acc:.4f} (Best!)")
    else:
        print(f"| Epoch {epoch+1:02} | Loss {avg_loss:.4f} | Dev Acc {acc:.4f}")

# ==========================================
# 6. 최종 평가 & 오답 분석 (Length Fixed)
# ==========================================
print("\n🔄 최고 성능 모델 로드 중...")
with open('best_model.pkl', 'rb') as f:
    best_params = pickle.load(f)
for i, param in enumerate(best_params):
    model.params[i][...] = param

# Test
model.set_train_flg(False)
test_score = model.predict(x_test)
test_pred = np.argmax(test_score, axis=1)
test_acc = np.sum(test_pred == t_test) / len(t_test)
print(f"🏆 Final Test Accuracy: {test_acc*100:.2f}%")

# 오답 분석 (길이 제한 적용)
def analyze_errors_fixed(model, df, vocab, dataset_name="Dev", max_len=300):
    error_records = []
    model.set_train_flg(False)
    label_map = {'TD': 0, 'SLI': 1}
    inv_label_map = {0: 'TD', 1: 'SLI'}

    print(f"\n📊 {dataset_name} Set 오답 분석 중...")

    for idx, row in df.iterrows():
        try:
            utts = extract_utterances(row['filename'], ['CHI'])
            if not utts: continue
            text = " ".join([u.clean_text for u in utts]).lower()
            tokens = text.split()
            ids = [vocab.get(w, vocab['<unk>']) for w in tokens]

            # [중요] Truncation 적용
            if len(ids) > max_len: ids = ids[:max_len]
            padded_ids = np.zeros(max_len, dtype=int)
            length = min(len(ids), max_len)
            padded_ids[:length] = ids[:length]

            x_in = padded_ids.reshape(1, -1)
            if np.__name__ == 'cupy': x_in = np.asarray(x_in)

            score = model.predict(x_in)
            if np.__name__ == 'cupy': score = np.asnumpy(score)

            pred_idx = np.argmax(score, axis=1)[0]
            true_idx = label_map[row['group']]

            if pred_idx != true_idx:
                error_records.append({
                    'Index': idx,
                    'Gender': row['gender'],
                    'Age': row['age'],
                    'True_Label': row['group'],
                    'Pred_Label': inv_label_map[pred_idx],
                    'Word_Count': len(tokens)
                })
        except: continue
    return pd.DataFrame(error_records)

dev_errors = analyze_errors_fixed(model, pd.read_csv('split/gillam_dev.csv'), vocab, "Dev", MAX_LEN)
test_errors = analyze_errors_fixed(model, pd.read_csv('split/gillam_test.csv'), vocab, "Test", MAX_LEN)
all_errors = pd.concat([dev_errors, test_errors])

print(f"\n총 오답 개수: {len(all_errors)} (Dev: {len(dev_errors)}, Test: {len(test_errors)})")
if not all_errors.empty:
    print("\n[오답 통계]")
    print(all_errors['Gender'].value_counts())
    print(all_errors['Age'].value_counts().sort_index())

✅ 라이브러리 로드 성공.
⏳ Vocab 생성 중...
✅ Vocab 크기: 3702
📂 데이터 로딩: split/gillam_train.csv
📂 데이터 로딩: split/gillam_dev.csv
📂 데이터 로딩: split/gillam_test.csv

🚀 학습 시작 (Bi-LSTM + MLP + R-Drop + FGM)
| Epoch 01 | Loss 0.6120 | Dev Acc 0.7794 (Best!)
| Epoch 02 | Loss 0.5033 | Dev Acc 0.7059
| Epoch 03 | Loss 0.4192 | Dev Acc 0.7206
| Epoch 04 | Loss 0.3481 | Dev Acc 0.8382 (Best!)
| Epoch 05 | Loss 0.2620 | Dev Acc 0.7353
| Epoch 06 | Loss 0.1667 | Dev Acc 0.7794
| Epoch 07 | Loss 0.1084 | Dev Acc 0.8529 (Best!)
| Epoch 08 | Loss 0.0769 | Dev Acc 0.7941
| Epoch 09 | Loss 0.0987 | Dev Acc 0.8235
| Epoch 10 | Loss 0.0607 | Dev Acc 0.8235
| Epoch 11 | Loss 0.0502 | Dev Acc 0.8676 (Best!)
| Epoch 12 | Loss 0.0142 | Dev Acc 0.8382
| Epoch 13 | Loss 0.0152 | Dev Acc 0.8235
| Epoch 14 | Loss 0.0336 | Dev Acc 0.8676
| Epoch 15 | Loss 0.0378 | Dev Acc 0.8824 (Best!)
| Epoch 16 | Loss 0.0225 | Dev Acc 0.8382
| Epoch 17 | Loss 0.0692 | Dev Acc 0.8971 (Best!)
| Epoch 18 | Loss 0.0193 | Dev Acc 0.8235
| Epoch 19 |